In [8]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters langchain-chroma langchain-core langchain-upstage
%pip install -U langchain langchainhub --quiet
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=300, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

# 문서를 가져와서,
loader = Docx2txtLoader('./tax_with_markdown.docx')

# Chunk 단위로 문서를 쪼갠다
document_list = loader.load_and_split(
    text_splitter=text_splitter
)

In [31]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatUpstage()
#llm = ChatOpenAI(model='gpt-4o')

embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

- Chroma : Vector in-memory Database

In [32]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-markdown-index"

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=100, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

chunked_documents = text_splitter.split_documents(document_list)

database = PineconeVectorStore.from_documents(
    documents=[],
    embedding=embedding,
    index_name=index_name
)

batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size : {batch_size}')
    batch = chunked_documents[i: i + batch_size]
    database.add_documents(batch)

index: 0, batch size : 100
index: 100, batch size : 100
index: 200, batch size : 100
index: 300, batch size : 100
index: 400, batch size : 100


In [93]:
query = '연봉 5천만원 초과인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query, k = 3)
print(retrived_docs)

[Document(id='b8c54eef-55da-49dd-b3ec-e02b8d2944ce', metadata={'source': './tax_with_markdown.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제4절 세액의 계산 <개정 2009. 12. 31.>\n\n\n\n제1관 세율 <개정 2009. 12. 31.>\n\n\n\n제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 

In [55]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [60]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs={"prompt" : prompt}
)

ai_message = qa_chain.invoke({"query" : query})
print(ai_message)

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?', 'result': '연봉 5천만원인 거주자의 소득세를 계산하려면 먼저 종합소득 과세표준을 구해야 합니다. 연봉에서 근로소득공제를 빼면 종합소득 과세표준이 되는데, 연봉 5천만원의 경우 약 3,400만원 정도가 됩니다. 이를 과세표준에 따른 세율에 적용하면 소득세는 약 360만원 정도 나옵니다. 하지만 이는 기본적인 계산 결과이고, 실제 납부세액은 각종 공제 및 세액 감면 등에 따라 달라질 수 있습니다.'}


In [90]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
당신은 대한민국 세법 전문가입니다. 아래 제공된 <context>의 내용을 바탕으로 사용자의 질문에 답변하세요.

[지침]
1. <context>에 표나 구체적인 계산 공식이 있다면 반드시 그 공식을 사용하여 답변하세요.
2. 금액 계산이 필요한 경우, 가능한 범위 내에서 직접 계산 예시를 보여주세요.
3. 문맥에 근거가 없는 내용은 추측하여 답변하지 마세요.

<context>
{context}
</context>

질문: {input}

""")

combine_docs_chain = create_stuff_documents_chain(llm, prompt)

retriever = database.as_retriever()
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

response = rag_chain.invoke({"input": query})
print(response["answer"])

연봉 5천만원을 초과하는 직장인의 소득세를 계산하기 위해서는 다음의 단계를 따릅니다.

1. 연봉에서 근로소득공제를 적용하여 종합소득 과세표준을 계산합니다.
2. 종합소득 과세표준에 따라 종합소득세율을 적용하여 산출세액을 계산합니다.

우선, 연봉 5천만원 근로자의 근로소득공제를 계산해봅시다.

제59조(근로소득세액공제) ① 근로소득이 있는 거주자에 대해서는 그 근로소득에 대한 종합소득산출세액에서 다음의 금액을 공제한다.

| 총급여액           | 공제금액                                 |
|-----------------|---------------------------------------|
| 3천만원 이하     | 70만원                                 |
| 3천만원 초과 7천만원 이하 | 70만원 - [(총급여액 - 3천만원) × 8/1000] |
| 7천만원 초과 1억2천만원 이하 | 66만원 - [(총급여액 - 7천만원) × 1/2]    |
| 1억2천만원 초과   | 50만원 - [(총급여액 - 1억2천만원) × 1/2] |

연봉 5천만원 근로자의 경우, 공제금액은 다음과 같습니다.

\[ 66만원 - [(5천만원 - 7천만원) \times 1/2] = 66만원 \]

(연봉 5천만원은 3천만원 초과 7천만원 이하의 구간에 해당하지만, 공제금액이 70만원보다 적으므로 70만원을 적용합니다.)

다음으로, 종합소득 과세표준에 따라 산출세액을 계산합니다.

제55조(세율) ① 거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다.

| 종합소득 과세표준          | 세율                                         |
|-------------------|--------------------------------------------|
| 1,40

In [94]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에, 질문만 답변해주세요.

사전 : {dictionary}

질문 : {{question}}

""")

dictionary_chain = prompt | llm | StrOutputParser()

In [95]:
new_question = dictionary_chain.invoke({"question": query})
new_question

"질문 : 연봉 5천만원 초과인 사람의 종합소득세는 얼마인가요?\n\n설명: 사용자의 질문은 이미 '사람을 나타내는 표현'이 '거주자'로 적절히 표현되어 있습니다. 따라서 변경할 필요가 없다고 판단하여 질문을 그대로 두었습니다. 그러나, 사전의 요구사항에 따라 변경이 필요하다면, '거주자'를 '사람'으로 변경할 수 있습니다."

In [91]:
#tax_chain = {"query" : dictionary_chain} | qa_chain
tax_chain = {"input" : dictionary_chain} | rag_chain
ai_response = tax_chain.invoke({"question": query})

In [92]:
ai_response

{'input': '연봉 5천만원 초과인 거주자의 소득세는 얼마인가요?',
 'context': [Document(id='b8c54eef-55da-49dd-b3ec-e02b8d2944ce', metadata={'source': './tax_with_markdown.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제4절 세액의 계산 <개정 2009. 12. 31.>\n\n\n\n제1관 세율 <개정 2009. 12. 31.>\n\n\n\n제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하